# Setup

## Parse trajectory

In [ ]:
import pathlib

import numpy as np

from analyzer.io.trajectory_parser import parse_trajectory

trajectory = parse_trajectory(pathlib.Path("demos/crossing/CROSSING_90_D_6.txt"), 16.0)

## Define geometry

### Define geometry in code

In [ ]:
import pygeos
from analyzer.data.geometry import Geometry

geometry = Geometry(
    pygeos.polygons(
        [
            (-9, 3),
            (-2, 3),
            (-2, 10),
            (2, 10),
            (2, 3),
            (9, 3),
            (9, -1),
            (2, -1),
            (2, -8),
            (-2, -8),
            (-2, -1),
            (-9, -1),
            (-9, 3),
        ]
    )
)

### Define geometry in file

The same format as for JuPedSim geometries can be used as well directly from files.

**Note:** Currently obstacles will not be parsed from the file!

In [ ]:
import pathlib
from analyzer.data.geometry import Geometry
from analyzer.io.geometry_parser import parse_geometry

# geometry = parse_geometry(pathlib.Path("demos/crossing/geometry.xml"))

## Define measurement areas and lines

In [ ]:
import pygeos

measurement_area = pygeos.polygons([(-2, -1), (-2, 3), (2, 3), (2, -1)])

measurement_line_left = pygeos.linestrings([(-3, 3), (-3, -1)])
measurement_line_right = pygeos.linestrings([(3, 3), (3, -1)])

measurement_line_top = pygeos.linestrings([(-2, 4), (2, 4)])
measurement_line_bottom = pygeos.linestrings([(-2, -2), (2, -2)])

## Plot trajectories, geometry, and measurement areas/lines

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(15, 20))
ax1 = fig.add_subplot(111, aspect="equal")

# Plot geometry
ax1.plot(*pygeos.to_shapely(geometry.walkable_area).exterior.xy, color="k")

## Plot measurement area
ax1.plot(*pygeos.to_shapely(measurement_area).exterior.xy, color="g")
ax1.fill(*pygeos.to_shapely(measurement_area).exterior.xy, color="g", alpha=0.3)

## Plot measurement lines
ax1.plot(*pygeos.to_shapely(measurement_line_left).xy, color="b")
ax1.plot(*pygeos.to_shapely(measurement_line_right).xy, color="b")
ax1.plot(*pygeos.to_shapely(measurement_line_top).xy, color="orange")
ax1.plot(*pygeos.to_shapely(measurement_line_bottom).xy, color="orange")

# Plot trajectories
for id, ped in trajectory.data.groupby("ID"):
    p = ax1.plot(ped["X"], ped["Y"], label=id, alpha=0.1, color="r")
    ax1.scatter(
        ped[ped.frame == ped.frame.max()]["X"],
        ped[ped.frame == ped.frame.max()]["Y"],
        c=p[-1].get_color(),
        marker="x",
    )
plt.show()

# Methods A

## Compute n-t and flow

In [ ]:
from analyzer.methods.flow_calculator import compute_n_t
from analyzer.methods.flow_calculator import compute_flow

from analyzer.methods.velocity_calculator import compute_individual_velocity

individual_speed = compute_individual_velocity(trajectory.data, trajectory.frame_rate, 10)

nt_left, crossing_left = compute_n_t(trajectory.data, measurement_line_left, trajectory.frame_rate)
nt_right, crossing_right = compute_n_t(
    trajectory.data, measurement_line_right, trajectory.frame_rate
)

delta_t = 100
flow_left = compute_flow(nt_left, crossing_left, individual_speed, delta_t, trajectory.frame_rate)
flow_right = compute_flow(
    nt_right, crossing_right, individual_speed, delta_t, trajectory.frame_rate
)

## Plot n-t diagram

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(7, 7))
ax1 = fig.add_subplot(111)

ax1.plot(nt_left.index, nt_left["Cumulative pedestrians"], label="left")
ax1.plot(nt_right.index, nt_right["Cumulative pedestrians"], label="right")

ax1.legend()
plt.show()

# Method B

## Compute individual velocity and density while passing the measurement area

In [ ]:
from analyzer.methods.velocity_calculator import compute_passing_speed
from analyzer.methods.density_calculator import compute_passing_density, compute_classic_density
from analyzer.methods.method_utils import compute_frame_range_in_area

density_per_frame = compute_classic_density(trajectory.data, measurement_area)
frames_in_area = compute_frame_range_in_area(trajectory.data, measurement_area)
passing_speed = compute_passing_speed(frames_in_area, trajectory.frame_rate)
passing_density = compute_passing_density(density_per_frame, frames_in_area)

## Plot fundamental diagram

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(7, 7))
ax1 = fig.add_subplot(111)

ax1.scatter(passing_density["density"], passing_speed["speed"])
ax1.set_xlim(left=0, right=5)
ax1.set_ylim(bottom=-0.2, top=3)

ax1.set_xlabel("rho / 1/m^2")
ax1.set_ylabel("v/ m/s")
ax1.grid()
plt.show()

# Method C

## Compute density and velocity

In [ ]:
from analyzer.methods.density_calculator import compute_classic_density
from analyzer.methods.velocity_calculator import compute_mean_velocity_per_frame
from analyzer.methods.method_utils import get_peds_in_area

peds_in_area = get_peds_in_area(trajectory.data, measurement_area)
mean_speed_area, individual_speed_area = compute_mean_velocity_per_frame(
    peds_in_area, trajectory.frame_rate, 5
)

classic_density = compute_classic_density(peds_in_area, measurement_area)

## Plot velocity, density and fundamental diagram

In [ ]:
import matplotlib.pyplot as plt

fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, figsize=(20, 6))

ax0.set_title("Velocity")
ax0.scatter(mean_speed_area.index, mean_speed_area, alpha=0.5, marker="x")
ax0.set_xlim(
    left=0,
)
ax0.set_ylim(bottom=-0.2, top=3)
ax0.set_xlabel("frame")
ax0.set_ylabel("v / m/s")
ax0.grid()

ax1.set_title("Density")
ax1.scatter(classic_density.index, classic_density["classic density"], alpha=0.5, marker="x")
ax1.set_xlim(left=0)
ax1.set_ylim(bottom=0, top=4)
ax1.set_xlabel("frame")
ax1.set_ylabel("rho / 1/m^2")
ax1.grid()

ax2.set_title("Fundamental Diagram")

ax2.scatter(
    classic_density["classic density"],
    mean_speed_area,
    alpha=0.2,
    marker="x",
)
ax2.set_aspect("equal")
ax2.set_xlim(left=0, right=5)
ax2.set_ylim(bottom=-0.2, top=3)

ax2.set_xlabel("rho / 1/m^2")
ax2.set_ylabel("v/ m/s")
ax2.grid()

# Method D

## Compute density and velocity

### Without cutoff radius

In [ ]:
from analyzer.methods.velocity_calculator import compute_voronoi_velocity
from analyzer.methods.density_calculator import compute_voronoi_density

voronoi_density, individual = compute_voronoi_density(trajectory.data, measurement_area, geometry)

voronoi_velocity, individual_velocity = compute_voronoi_velocity(
    trajectory.data, individual, trajectory.frame_rate, 5, measurement_area
)

### With cutoff radius

In [ ]:
from analyzer.methods.velocity_calculator import compute_voronoi_velocity
from analyzer.methods.density_calculator import compute_voronoi_density

voronoi_density_cutoff, individual_cutoff = compute_voronoi_density(
    trajectory.data, measurement_area, geometry, 0.8, 12
)

voronoi_velocity_cutoff, individual_velocity_cutoff = compute_voronoi_velocity(
    trajectory.data, individual, trajectory.frame_rate, 5, measurement_area
)

## Compute velocity and density profiles

In [ ]:
import pandas as pd

from analyzer.methods.profile_calculator import compute_profiles
from analyzer.methods.velocity_calculator import compute_voronoi_velocity
from analyzer.methods.density_calculator import compute_voronoi_density

frames_data = trajectory.data[trajectory.data.frame.isin(range(800, 1000))]

voronoi_density_frames, individual_frames = compute_voronoi_density(
    frames_data, measurement_area, geometry, 0.8, 12
)
voronoi_velocity_frames, individual_velocity_frames = compute_voronoi_velocity(
    trajectory.data, individual_frames, trajectory.frame_rate, 5, measurement_area
)

density_profiles, velocity_profiles = compute_profiles(
    pd.merge(individual_frames, individual_velocity_frames, on=["ID", "frame"], how="left"),
    geometry.walkable_area,
    0.2,
)

## Plots

### Plot velocity, density and fundamental diagram

In [ ]:
import matplotlib.pyplot as plt

for velocity, density, title in [
    (voronoi_velocity, voronoi_density, "without cutoff"),
    (voronoi_velocity_cutoff, voronoi_density_cutoff, "with cutoff"),
]:
    fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, figsize=(20, 6))
    fig.suptitle(title)
    ax0.set_title("Velocity")
    ax0.scatter(velocity.index, velocity, alpha=0.5, marker="x")
    ax0.set_xlim(
        left=0,
    )
    ax0.set_ylim(bottom=-0.2, top=3)
    ax0.set_xlabel("frame")
    ax0.set_ylabel("v / m/s")
    ax0.grid()

    ax1.set_title("Density")
    ax1.scatter(density.index, density["voronoi density"], alpha=0.5, marker="x")
    ax1.set_xlim(left=0)
    ax1.set_ylim(bottom=0, top=4)
    ax1.set_xlabel("frame")
    ax1.set_ylabel("rho / 1/m^2")
    ax1.grid()

    ax2.set_title("Fundamental Diagram")

    ax2.scatter(
        density["voronoi density"],
        velocity,
        alpha=0.2,
        marker="x",
    )
    ax2.set_aspect("equal")
    ax2.set_xlim(left=0, right=5)
    ax2.set_ylim(bottom=-0.2, top=3)

    ax2.set_xlabel("rho / 1/m^2")
    ax2.set_ylabel("v/ m/s")
    ax2.grid()

### Plot voronoi cells

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

frame = 1000
use_cut_off_data = True
data = individual_cutoff if use_cut_off_data else individual

data = pd.merge(trajectory.data, data, on=["ID", "frame"], how="left")
data = data[data.frame == frame]


fig = plt.figure(figsize=(15, 20))
ax1 = fig.add_subplot(111, aspect="equal")

# Plot geometry
ax1.plot(*pygeos.to_shapely(geometry.walkable_area).exterior.xy, color="k")

# Plot measurement area
ax1.plot(*pygeos.to_shapely(measurement_area).exterior.xy, color="g")

# Plot voronoi cells
for id, ped in data.groupby("ID"):
    if not pygeos.is_empty(ped["intersection voronoi"].iat[0]):
        intersection_polygon = ped["intersection voronoi"]
        poly = pygeos.to_shapely(intersection_polygon)[0]
        p = ax1.plot(*poly.exterior.xy, alpha=0.5)
        ax1.fill(*poly.exterior.xy, fc=p[-1].get_color(), alpha=0.2)

        intersection_polygon = ped["individual voronoi"]
        poly = pygeos.to_shapely(intersection_polygon)[0]
        ax1.plot(*poly.exterior.xy, alpha=0.2, color=p[-1].get_color())
        ax1.fill(*poly.exterior.xy, fc=p[-1].get_color(), alpha=0.1)

        ax1.scatter(ped["X"], ped["Y"], c=p[-1].get_color())

    else:
        intersection_polygon = ped["individual voronoi"]
        poly = pygeos.to_shapely(intersection_polygon)[0]
        ax1.plot(*poly.exterior.xy, alpha=0.2, color="gray")
        ax1.fill(*poly.exterior.xy, fc="gray", alpha=0.02)

        ax1.scatter(ped["X"], ped["Y"], color="k", alpha=0.1)

### Plot profiles

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

bounds = pygeos.bounds(geometry.walkable_area)

fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(10, 10))

ax0.set_title("Density")
cm = ax0.imshow(
    np.mean(density_profiles, axis=0),
    extent=[bounds[0], bounds[2], bounds[1], bounds[3]],
    interpolation="None",
    cmap="RdBu_r",
    vmin=0,
    vmax=5,
)
fig.colorbar(cm, ax=ax0, shrink=0.3)
ax0.plot(*pygeos.to_shapely(geometry.walkable_area).exterior.xy, color="k")


ax1.set_title("Velocity")
cm = ax1.imshow(
    np.mean(velocity_profiles, axis=0),
    extent=[bounds[0], bounds[2], bounds[1], bounds[3]],
    cmap="RdBu",
    vmin=0,
    vmax=1.5,
)
fig.colorbar(cm, ax=ax1, shrink=0.3)

ax1.plot(*pygeos.to_shapely(geometry.walkable_area).exterior.xy, color="k")

fig.tight_layout()

In [ ]:
import pandas as pd

from analyzer.methods.profile_calculator import compute_profiles
from analyzer.methods.velocity_calculator import compute_voronoi_velocity
from analyzer.methods.density_calculator import compute_voronoi_density

frames_data = trajectory.data[trajectory.data.frame.isin(range(800, 1000))]

voronoi_density_frames, individual_frames = compute_voronoi_density(
    frames_data, measurement_area, geometry, 0.8, 12
)
voronoi_velocity_frames, individual_velocity_frames = compute_voronoi_velocity(
    trajectory.data, individual_frames, trajectory.frame_rate, 5, measurement_area
)

density_profiles, velocity_profiles = compute_profiles(
    pd.merge(individual_frames, individual_velocity_frames, on=["ID", "frame"], how="left"),
    geometry.walkable_area,
    0.2,
)

## Plots

### Plot velocity, density and fundamental diagram

In [ ]:
import matplotlib.pyplot as plt

for velocity, density, title in [
    (voronoi_velocity, voronoi_density, "without cutoff"),
    (voronoi_velocity_cutoff, voronoi_density_cutoff, "with cutoff"),
]:
    fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, figsize=(20, 6))
    fig.suptitle(title)
    ax0.set_title("Velocity")
    ax0.scatter(velocity.index, velocity, alpha=0.5, marker="x")
    ax0.set_xlim(
        left=0,
    )
    ax0.set_ylim(bottom=-0.2, top=3)
    ax0.set_xlabel("frame")
    ax0.set_ylabel("v / m/s")
    ax0.grid()

    ax1.set_title("Density")
    ax1.scatter(density.index, density["voronoi density"], alpha=0.5, marker="x")
    ax1.set_xlim(left=0)
    ax1.set_ylim(bottom=0, top=4)
    ax1.set_xlabel("frame")
    ax1.set_ylabel("rho / 1/m^2")
    ax1.grid()

    ax2.set_title("Fundamental Diagram")

    ax2.scatter(
        density["voronoi density"],
        velocity,
        alpha=0.2,
        marker="x",
    )
    ax2.set_aspect("equal")
    ax2.set_xlim(left=0, right=5)
    ax2.set_ylim(bottom=-0.2, top=3)

    ax2.set_xlabel("rho / 1/m^2")
    ax2.set_ylabel("v/ m/s")
    ax2.grid()

### Plot voronoi cells

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

frame = 1000
use_cut_off_data = True
data = individual_cutoff if use_cut_off_data else individual

data = pd.merge(trajectory.data, data, on=["ID", "frame"], how="left")
data = data[data.frame == frame]


fig = plt.figure(figsize=(15, 20))
ax1 = fig.add_subplot(111, aspect="equal")

# Plot geometry
ax1.plot(*pygeos.to_shapely(geometry.walkable_area).exterior.xy, color="k")

# Plot measurement area
ax1.plot(*pygeos.to_shapely(measurement_area).exterior.xy, color="g")

# Plot voronoi cells
for id, ped in data.groupby("ID"):
    if not pygeos.is_empty(ped["intersection voronoi"].iat[0]):
        intersection_polygon = ped["intersection voronoi"]
        poly = pygeos.to_shapely(intersection_polygon)[0]
        p = ax1.plot(*poly.exterior.xy, alpha=0.5)
        ax1.fill(*poly.exterior.xy, fc=p[-1].get_color(), alpha=0.2)

        intersection_polygon = ped["individual voronoi"]
        poly = pygeos.to_shapely(intersection_polygon)[0]
        ax1.plot(*poly.exterior.xy, alpha=0.2, color=p[-1].get_color())
        ax1.fill(*poly.exterior.xy, fc=p[-1].get_color(), alpha=0.1)

        ax1.scatter(ped["X"], ped["Y"], c=p[-1].get_color())

    else:
        intersection_polygon = ped["individual voronoi"]
        poly = pygeos.to_shapely(intersection_polygon)[0]
        ax1.plot(*poly.exterior.xy, alpha=0.2, color="gray")
        ax1.fill(*poly.exterior.xy, fc="gray", alpha=0.02)

        ax1.scatter(ped["X"], ped["Y"], color="k", alpha=0.1)

### Plot profiles

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

bounds = pygeos.bounds(geometry.walkable_area)

fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(10, 10))

ax0.set_title("Density")
cm = ax0.imshow(
    np.mean(density_profiles, axis=0),
    extent=[bounds[0], bounds[2], bounds[1], bounds[3]],
    interpolation="None",
    cmap="RdBu_r",
    vmin=0,
    vmax=5,
)
fig.colorbar(cm, ax=ax0, shrink=0.3)
ax0.plot(*pygeos.to_shapely(geometry.walkable_area).exterior.xy, color="k")


ax1.set_title("Velocity")
cm = ax1.imshow(
    np.mean(velocity_profiles, axis=0),
    extent=[bounds[0], bounds[2], bounds[1], bounds[3]],
    cmap="RdBu",
    vmin=0,
    vmax=1.5,
)
fig.colorbar(cm, ax=ax1, shrink=0.3)

ax1.plot(*pygeos.to_shapely(geometry.walkable_area).exterior.xy, color="k")

fig.tight_layout()